<h1 align=center><font size = 5>Choose Neighborhoods in Toronto to Open a Sushi Restaurant</font></h1>

## Introduction

This project demonstrates a data-driven approach to choose potential neighborhoods in Toronto in which we can open a Sushi restaurant. First, I conduct exploratory analysis on two North American cities, New York City and Toroto. The main focus is on the distribuition of business venues over the neigborhoods. Second, I cluster Neighborhoods with Sushi restaurants in New York City to their counterparts without Sushi restaurants yet in Toronto. In the end, we can choose those neighborhoods in Toronto similar as New York City to open a Sushi Restaurant.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Explore Neighborhoods in Toronto</a>

2.  <a href="#item2">Explore Neighborhoods in New York City</a>
    
3.  <a href="#item3">Select Relevant Neighborhoods from New York City and Toronto</a>

4.  <a href="#item4">Neighborhood Clustering</a>
    
5.  <a href="#item5">Examine Clusters</a>
    
6.  <a href="#item6">Conclusions</a>
    </font>
    </div>


## 1. Explore Neighborhoods in Toronto  <a class="anchor" id="item1"></a>


scrape data on wiki pages

In [1]:
import pandas as pd 
import requests
!pip install bs4
!pip install lxml
from bs4 import BeautifulSoup

 

req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") 

soup = BeautifulSoup(req.content,'lxml')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


/home/zhilin/code/private/coursera-data-sci-capstone/venv-py37/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


You are using pip version 19.0.3, however version 21.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 21.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


make lat/long-itude columns using csv


In [3]:
import os


data_path = "data"
file_name = "Geospatial_Coordinates.csv"
csv_file = os.path.join(os.getcwd(), data_path, file_name)

# read la/long-titude csv
df_lalong = pd.read_csv(csv_file, names=["PostalCode", "Latitude", "Longitude"])


# join table according to colomn: Postal Code
df_joined = df.join(df_lalong.set_index('PostalCode'), on='PostalCode')
# print(df_joined["Latitude"].max())
# print(df_joined["Latitude"].min())
# print(df_joined["Longitude"].max())
# print(df_joined["Longitude"].min())
df_joined

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.4647633
4,M7A,Queen's Park,Ontario Provincial Government,43.6623015,-79.3894938
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6536536,-79.5069436
99,M4Y,Downtown Toronto,Church and Wellesley,43.6658599,-79.3831599
100,M7Y,East Toronto Business,Enclave of M4L,43.6627439,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6362579,-79.4985091


In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_joined['Borough'].unique()),
        df_joined.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


get Toronto geo info

In [5]:
import folium
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors



address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


visualize neighborhoods geo info on a map

In [6]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# convert lat/long-itude str data into float so that it can be rendered on map
df_joined['Latitude'] = df_joined['Latitude'].astype(float)
df_joined['Longitude'] = df_joined['Longitude'].astype(float)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_joined['Latitude'], df_joined['Longitude'], df_joined['Borough'], df_joined['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## clustering neighborhoods according to venues

get venues data from foursquare

In [7]:
import os 


CLIENT_ID = os.getenv("CLIENT_ID") # your Foursquare ID
CLIENT_SECRET = os.getenv("CLIENT_SECRET") # your Foursquare Secret
VERSION = '20210605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value


In [8]:
neighborhood_latitude = df_joined.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_joined.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_joined.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [9]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)


In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60d19c6627addc716113e063'},
 'response': {'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 29,
  'suggestedBounds': {'ne': {'lat': 43.762258609000014,
    'lng': -79.31721997969855},
   'sw': {'lat': 43.74425859099999, 'lng': -79.34209302030145}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b8991cbf964a520814232e3',
       'name': "Allwyn's Bakery",
       'location': {'address': '81 Underhill drive',
        'lat': 43.75984035203157,
        'lng': -79.32471879917513,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.75984035203157,
          'lng': -79.32471879917513}],
        'distance': 833,
    

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/zhilin/code/private/coursera-data-sci-capstone/venv-py37/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,name,categories,lat,lng
0,Allwyn's Bakery,Caribbean Restaurant,43.759840,-79.324719
1,Bruno's valu-mart,Grocery Store,43.746143,-79.324630
2,Brookbanks Park,Park,43.751976,-79.332140
3,A&W,Fast Food Restaurant,43.760643,-79.326865
4,Shoppers Drug Mart,Pharmacy,43.745315,-79.325800


In [13]:
nearby_venues.shape

(29, 4)

explore each neighborhood

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
toronto_venues = getNearbyVenues(names=df_joined['Neighborhood'],
                                   latitudes=df_joined['Latitude'],
                                   longitudes=df_joined['Longitude'],
                                   radius=1000
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [16]:
print(toronto_venues.shape)
toronto_venues.head()

(4740, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store
2,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
3,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Parkwoods,43.753259,-79.329656,Shoppers Drug Mart,43.745315,-79.325800,Pharmacy


In [17]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,48,48,48,48,48,48
"Alderwood, Long Branch",24,24,24,24,24,24
"Bathurst Manor, Wilson Heights, Downsview North",29,29,29,29,29,29
Bayview Village,17,17,17,17,17,17
"Bedford Park, Lawrence Manor East",40,40,40,40,40,40
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",24,24,24,24,24,24
Woburn,9,9,9,9,9,9
Woodbine Heights,28,28,28,28,28,28


find unique venues

In [18]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 323 uniques categories.


analyze each neighborhood

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
toronto_onehot.shape

(4740, 323)

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,...,0.0,0.000000,0.020833,0.0,0.0,0.0,0.000,0.000,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025000,0.0,...,0.0,0.025000,0.000000,0.0,0.0,0.0,0.025,0.025,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000,0.0,0.0
98,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000,0.0,0.0
99,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.035714,0.000000,0.0,0.0,0.0,0.000,0.000,0.0,0.0
100,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000,0.000,0.0,0.0


In [22]:
toronto_grouped.shape

(102, 323)

select the top N venues in neighborhoods

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
toronto_neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Caribbean Restaurant,Pizza Place,Bakery,Supermarket,Discount Store,Clothing Store,Lounge,Malay Restaurant
1,"Alderwood, Long Branch",Discount Store,Pizza Place,Park,Playground,Shopping Mall,Garden Center,Gas Station,Liquor Store,Sandwich Place,Moroccan Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Pizza Place,Park,Coffee Shop,Ski Area,Bridal Shop,Fried Chicken Joint,Frozen Yogurt Shop,Restaurant,Shopping Mall
3,Bayview Village,Bank,Gas Station,Grocery Store,Intersection,Japanese Restaurant,Restaurant,Trail,Playground,Chinese Restaurant,Skating Rink
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Bank,Restaurant,Fast Food Restaurant,Sushi Restaurant,Pizza Place,Greek Restaurant,Grocery Store,Liquor Store


clustering toronto neighborhoods according to venues

In [25]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
toronto_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
toronto_kmeans.labels_[0:10] 

array([2, 2, 2, 2, 1, 1, 2, 1, 1, 2], dtype=int32)

In [26]:
# add clustering labels
toronto_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', toronto_kmeans.labels_)
toronto_merged = df_joined[:]
# remove neighborhood "Upper Rouge", because it contains no data
toronto_merged = toronto_merged[toronto_merged["Neighborhood"] != "Upper Rouge"].reset_index()
# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
# toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head() # check the last columns!

,index,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Park,Bus Stop,Shopping Mall,Pharmacy,Coffee Shop,Café,Supermarket,Tennis Court,Shop & Service,Grocery Store
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,4,Coffee Shop,Sporting Goods Shop,Intersection,Men's Store,Lounge,Grocery Store,Park,Gym / Fitness Center,Golf Course,Pizza Place
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Sushi Restaurant,Café,Restaurant,Bakery,Park,Pizza Place,Sandwich Place,Italian Restaurant
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4,Fast Food Restaurant,Restaurant,Clothing Store,Coffee Shop,Furniture / Home Store,Vietnamese Restaurant,Sushi Restaurant,Fried Chicken Joint,Seafood Restaurant,Paper / Office Supplies Store
4,4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Park,Pizza Place,Dance Studio,Gym,Café,Clothing Store,Breakfast Spot


In [27]:
toronto_merged.shape

(102, 17)

In [28]:
toronto_neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Agincourt,Chinese Restaurant,Shopping Mall,Caribbean Restaurant,Pizza Place,Bakery,Supermarket,Discount Store,Clothing Store,Lounge,Malay Restaurant
1,2,"Alderwood, Long Branch",Discount Store,Pizza Place,Park,Playground,Shopping Mall,Garden Center,Gas Station,Liquor Store,Sandwich Place,Moroccan Restaurant
2,2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Pizza Place,Park,Coffee Shop,Ski Area,Bridal Shop,Fried Chicken Joint,Frozen Yogurt Shop,Restaurant,Shopping Mall
3,2,Bayview Village,Bank,Gas Station,Grocery Store,Intersection,Japanese Restaurant,Restaurant,Trail,Playground,Chinese Restaurant,Skating Rink
4,1,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Bank,Restaurant,Fast Food Restaurant,Sushi Restaurant,Pizza Place,Greek Restaurant,Grocery Store,Liquor Store


visualize clusters on a map

In [29]:
# create map
toronto_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto_map_clusters)
       
toronto_map_clusters

examine clusters

In [30]:
cluster_num = 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == cluster_num, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,-79.329656,2,Park,Bus Stop,Shopping Mall,Pharmacy,Coffee Shop,Café,Supermarket,Tennis Court,Shop & Service,Grocery Store
5,M9A,-79.532242,2,Pharmacy,Bank,Convenience Store,Playground,Golf Course,Shopping Mall,Grocery Store,Skating Rink,Café,Park
11,M9B,-79.554724,2,Park,Hotel,Pizza Place,Fish & Chips Shop,Mexican Restaurant,Grocery Store,Restaurant,Bank,Gym,Clothing Store
12,M1C,-79.160497,2,Breakfast Spot,Playground,Burger Joint,Park,Italian Restaurant,Zoo,Music Venue,Noodle House,Nightclub,Newsagent
21,M6E,-79.453512,2,Park,Pharmacy,Bus Stop,Portuguese Restaurant,Bank,Bakery,Falafel Restaurant,Grocery Store,Coffee Shop,Fast Food Restaurant
22,M1G,-79.216917,2,Park,Coffee Shop,Mobile Phone Shop,Chinese Restaurant,Fast Food Restaurant,Indian Restaurant,Pharmacy,Zoo,Nail Salon,Office
27,M2H,-79.363452,2,Pharmacy,Coffee Shop,Park,Tennis Court,Recreation Center,Grocery Store,Korean Restaurant,Bank,Restaurant,Residential Building (Apartment / Condo)
28,M3H,-79.442259,2,Bank,Pizza Place,Park,Coffee Shop,Ski Area,Bridal Shop,Fried Chicken Joint,Frozen Yogurt Shop,Restaurant,Shopping Mall
39,M2K,-79.385975,2,Bank,Gas Station,Grocery Store,Intersection,Japanese Restaurant,Restaurant,Trail,Playground,Chinese Restaurant,Skating Rink
46,M3L,-79.506944,2,Park,Bank,Pizza Place,Shopping Mall,Grocery Store,Vietnamese Restaurant,Moving Target,Zoo,Nightclub,Newsagent


## 2. Explore Neighborhoods in New York City <a class="anchor" id="item2"></a>


In [31]:
import json


# load NY data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
ny_neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
ny_neighborhoods = pd.DataFrame(columns=column_names)
for data in ny_neighborhoods_data:
    ny_borough = ny_neighborhood_name = data['properties']['borough'] 
    ny_neighborhood_name = data['properties']['name']
        
    ny_neighborhood_latlon = data['geometry']['coordinates']
    ny_neighborhood_lat = ny_neighborhood_latlon[1]
    ny_neighborhood_lon = ny_neighborhood_latlon[0]
    
    ny_neighborhoods = ny_neighborhoods.append({'Borough': ny_borough,
                                          'Neighborhood': ny_neighborhood_name,
                                          'Latitude': ny_neighborhood_lat,
                                          'Longitude': ny_neighborhood_lon}, ignore_index=True)

# show data on map
ny_address = 'New York City'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(ny_address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of NY are {}, {}.'.format(latitude, longitude))

# create map of Manhattan using latitude and longitude values
map_ny = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ny_neighborhoods['Latitude'], ny_neighborhoods['Longitude'], ny_neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

The geograpical coordinate of NY are 40.7127281, -74.0060152.


In [186]:
ny_neighborhoods.shape

(306, 4)

In [32]:
# get foursqare data
ny_venues = getNearbyVenues(names=ny_neighborhoods['Neighborhood'],
                                   latitudes=ny_neighborhoods['Latitude'],
                                   longitudes=ny_neighborhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [33]:
ny_venues.shape

(9495, 7)

In [78]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
ny_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
ny_neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    ny_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

ny_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Supermarket,Deli / Bodega,Discount Store,Spa,Chinese Restaurant,Check Cashing Service,Grocery Store,Gas Station,Fried Chicken Joint
1,Annadale,Dance Studio,Train Station,Pharmacy,Liquor Store,Pizza Place,Cosmetics Shop,Bakery,Sushi Restaurant,Diner,Restaurant
2,Arden Heights,Deli / Bodega,Playground,Coffee Shop,Pizza Place,Pharmacy,Other Nightlife,Performing Arts Venue,Pet Store,Pet Service,Optical Shop
3,Arlington,Bus Stop,Deli / Bodega,Intersection,Paella Restaurant,Pet Store,Pet Service,Pet Café,Peruvian Restaurant,Persian Restaurant,Perfume Shop
4,Arrochar,Bus Stop,Italian Restaurant,Deli / Bodega,Food Truck,Middle Eastern Restaurant,Mediterranean Restaurant,Pizza Place,Hotel,Sandwich Place,Nail Salon


In [79]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
ny_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
ny_kmeans.labels_[0:10] 

array([0, 0, 0, 3, 0, 0, 0, 0, 0, 0], dtype=int32)

In [80]:
# add clustering labels
ny_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', ny_kmeans.labels_)
ny_merged = ny_neighborhoods[:]
ny_merged = ny_merged.join(ny_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
# toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
ny_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,0,Pharmacy,Deli / Bodega,Laundromat,Food,Caribbean Restaurant,Ice Cream Shop,Donut Shop,Sandwich Place,Dessert Shop,Outdoors & Recreation
1,Bronx,Co-op City,40.874294,-73.829939,0,Bus Station,Restaurant,Deli / Bodega,Donut Shop,Baseball Field,Pizza Place,Park,Grocery Store,Pharmacy,Post Office
2,Bronx,Eastchester,40.887556,-73.827806,0,Bus Station,Caribbean Restaurant,Diner,Pizza Place,Food & Drink Shop,Seafood Restaurant,Bowling Alley,Convenience Store,Cosmetics Shop,Bakery
3,Bronx,Fieldston,40.895437,-73.905643,0,Bus Station,Music Venue,Plaza,Yoga Studio,Paella Restaurant,Pet Service,Pet Café,Peruvian Restaurant,Persian Restaurant,Perfume Shop
4,Bronx,Riverdale,40.890834,-73.912585,0,Park,Gym,Gym / Fitness Center,Bus Station,Bank,Medical Supply Store,Playground,Plaza,Pet Café,Peruvian Restaurant


In [81]:
# create map
ny_map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(ny_map_clusters)
       
ny_map_clusters

In [41]:
ny_neighborhoods_venues_sorted.shape

(302, 12)

## 3. Select Relevant Neighborhoods from New York City and Toronto <a class="anchor" id="item3"></a>

find NY neighborhoods with sushi restaurants as the top 10 venues

In [68]:
df_ny_sushi = ny_neighborhoods_venues_sorted[ny_neighborhoods_venues_sorted.isin(["Sushi Restaurant"]).any(axis=1)]
ny_sushi_df = ny_venues.loc[ny_venues["Neighborhood"].isin(df_ny_sushi["Neighborhood"].tolist())]
ny_sushi_df.insert(0, 'City', 'NY')
ny_sushi_df.reset_index(drop=True, inplace=True)

find Toronto neighborhoods without sushi restaurants yet

In [69]:
no_sushi = toronto_grouped[toronto_grouped["Sushi Restaurant"] <= 0]
toronto_no_sushi_df = toronto_venues.loc[toronto_venues["Neighborhood"].isin(no_sushi["Neighborhood"].tolist())]
toronto_no_sushi_df.insert(0, 'City', 'Toronto')
toronto_no_sushi_df.reset_index(drop=True, inplace=True)

In [70]:
print("{} Toronto neighborhoods do NOT have sushi restaurants yet". format(
    len(toronto_no_sushi_df.groupby("Neighborhood").groups)))
print("{} NY neighborhoods have sushi restaurants as the TOP 10 venues". format(
    len(ny_sushi_df.groupby("Neighborhood").groups)))

59 Toronto neighborhoods do NOT have sushi restaurants yet
22 NY neighborhoods have sushi restaurants as the TOP 10 venues


concatenate two data frames

In [71]:
concat_df = pd.concat([toronto_no_sushi_df, ny_sushi_df])
concat_df

,City,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Toronto,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Toronto,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store
2,Toronto,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
3,Toronto,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant
4,Toronto,Parkwoods,43.753259,-79.329656,Shoppers Drug Mart,43.745315,-79.325800,Pharmacy
...,...,...,...,...,...,...,...,...
1192,NY,Prince's Bay,40.526264,-74.201526,MTA SIR - Prince's Bay,40.525290,-74.200576,Train Station
1193,NY,Prince's Bay,40.526264,-74.201526,Island Taste,40.525834,-74.201209,Chinese Restaurant
1194,NY,Prince's Bay,40.526264,-74.201526,Certo's Gourmet Foods & Caterers,40.525838,-74.201117,Italian Restaurant
1195,NY,Prince's Bay,40.526264,-74.201526,Savo Brothers,40.529062,-74.198113,Construction & Landscaping


In [97]:
# one hot encoding
concat_onehot = pd.get_dummies(concat_df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
# concat_onehot['City'] = concat_df['City'] 
concat_onehot['Neighborhood'] = concat_df['Neighborhood']
# move neighborhood column to the first column
fixed_columns = list(concat_onehot.columns[-1:]) + list(concat_onehot.columns[:-1])
concat_onehot = concat_onehot[fixed_columns]



concat_grouped = concat_onehot.groupby(['Neighborhood']).mean().reset_index()

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
concat_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
concat_neighborhoods_venues_sorted['Neighborhood'] = concat_grouped['Neighborhood']

for ind in np.arange(concat_grouped.shape[0]):
    concat_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(concat_grouped.iloc[ind, :], num_top_venues)

concat_neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood, Long Branch",Discount Store,Pizza Place,Park,Intersection,Shopping Mall,Grocery Store,Gas Station,Garden Center,Moroccan Restaurant,Pub
1,Annadale,Liquor Store,Pizza Place,Dance Studio,Cosmetics Shop,Restaurant,Food,Sushi Restaurant,Bakery,Diner,Pharmacy
2,Bayside,Bar,Ice Cream Shop,Mexican Restaurant,Sushi Restaurant,Spa,Pub,Pizza Place,Bakery,American Restaurant,Steakhouse
3,Bayview Village,Bank,Japanese Restaurant,Gas Station,Grocery Store,Intersection,Restaurant,Café,Chinese Restaurant,Park,Playground
4,Bensonhurst,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Pizza Place,Donut Shop,Ice Cream Shop,Dessert Shop,Cha Chaan Teng,Supermarket,Smoke Shop


## 4. Neighborhood Clustering <a class="anchor" id="item4"></a>

In [98]:
# clustering
# set number of clusters
kclusters = 6

df_sushi_clustering = concat_grouped.drop(['Neighborhood'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_sushi_clustering)

# add clustering labels
concat_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# NY
ny_sushi_neighborhoods = ny_neighborhoods.loc[ny_neighborhoods['Neighborhood'].isin(df_ny_sushi["Neighborhood"].tolist())]
ny_sushi_neighborhoods.drop(['Borough'], 1, inplace=True)
ny_sushi_neighborhoods.insert(0, 'City', 'NY')

# Toronto
toronto_no_sushi_neighborhoods = df_joined.loc[df_joined['Neighborhood'].isin(no_sushi["Neighborhood"].tolist())]
toronto_no_sushi_neighborhoods.drop(['Borough', 'PostalCode'], 1, inplace=True)
toronto_no_sushi_neighborhoods.insert(0, 'City', 'Toronto')

concat_neighborhoods = pd.concat([toronto_no_sushi_neighborhoods, ny_sushi_neighborhoods])
concat_neighborhoods.reset_index(drop=True, inplace=True)

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
concat_neighborhoods = concat_neighborhoods.join(concat_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

concat_neighborhoods.head() # check the last columns!

/home/zhilin/code/private/coursera-data-sci-capstone/venv-py37/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,City,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Toronto,Parkwoods,43.753259,-79.329656,3,Park,Bus Stop,Pharmacy,Shopping Mall,Convenience Store,Shop & Service,Coffee Shop,Fish & Chips Shop,Tennis Court,Fast Food Restaurant
1,Toronto,Victoria Village,43.725882,-79.315572,3,Coffee Shop,Gym / Fitness Center,Pizza Place,Men's Store,Golf Course,Park,Grocery Store,Lounge,Sporting Goods Shop,Hockey Arena
2,Toronto,Islington Avenue,43.667856,-79.532242,3,Pharmacy,Bank,Café,Grocery Store,Skating Rink,Golf Course,Playground,Park,Shopping Mall,Convenience Store
3,Toronto,"Malvern, Rouge",43.806686,-79.194353,3,Trail,Fast Food Restaurant,Caribbean Restaurant,Bakery,Chinese Restaurant,Sandwich Place,Bank,Arts & Crafts Store,Greek Restaurant,Coffee Shop
4,Toronto,Don Mills North,43.745906,-79.352188,3,Pizza Place,Coffee Shop,Japanese Restaurant,Supermarket,Gym,Mexican Restaurant,Thai Restaurant,Café,Shopping Mall,Bar


## 5. Examine Clusters <a class="anchor" id="item5"></a>

In [99]:
for i in range(kclusters):
    cluster = concat_neighborhoods.loc[concat_neighborhoods['Cluster Labels'] == i, concat_neighborhoods.columns[[0, 1] + list(range(5, concat_neighborhoods.shape[1]))]]
    if cluster[cluster["City"] == "Toronto"].shape[0] < cluster[cluster["City"] == "NY"].shape[0]:
        print("cluster {} has the majority NY".format(i))

cluster 1 has the majority NY


In [101]:
t_cluster = 1
cluster = concat_neighborhoods.loc[concat_neighborhoods['Cluster Labels'] == t_cluster, concat_neighborhoods.columns[[0, 1] + list(range(5, concat_neighborhoods.shape[1]))]]
cluster

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Toronto,"Little Portugal, Trinity",Restaurant,Italian Restaurant,Café,Vegetarian / Vegan Restaurant,Bar,Bakery,Cocktail Bar,Coffee Shop,Pizza Place,Asian Restaurant
59,NY,North Riverdale,Pizza Place,Italian Restaurant,Bank,Bagel Shop,Grocery Store,Mexican Restaurant,Mobile Phone Shop,Donut Shop,Sandwich Place,Coffee Shop
60,NY,Bensonhurst,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Pizza Place,Donut Shop,Ice Cream Shop,Dessert Shop,Cha Chaan Teng,Supermarket,Smoke Shop
61,NY,Brighton Beach,Russian Restaurant,Beach,Restaurant,Mobile Phone Shop,Bank,Eastern European Restaurant,Pharmacy,Sushi Restaurant,Donut Shop,Gourmet Shop
62,NY,Prospect Lefferts Gardens,Café,Pizza Place,Caribbean Restaurant,Chinese Restaurant,Sushi Restaurant,Supermarket,Bakery,Indian Restaurant,Garden,Grocery Store
63,NY,North Side,Coffee Shop,Pizza Place,Yoga Studio,Bakery,Wine Bar,Vegetarian / Vegan Restaurant,Sushi Restaurant,Shoe Store,Salon / Barbershop,Juice Bar
64,NY,South Side,Bar,Coffee Shop,American Restaurant,Pizza Place,Wine Bar,Yoga Studio,South American Restaurant,Caribbean Restaurant,Cycle Studio,Grocery Store
65,NY,Yorkville,Italian Restaurant,Coffee Shop,Sushi Restaurant,Gym,Japanese Restaurant,Bar,Diner,Deli / Bodega,Ice Cream Shop,Pizza Place
66,NY,Lenox Hill,Italian Restaurant,Café,Pizza Place,Sushi Restaurant,Bank,Gym / Fitness Center,Burger Joint,Cocktail Bar,Coffee Shop,Gym
67,NY,Midtown,Clothing Store,Hotel,Spa,Theater,Food Truck,Coffee Shop,Sushi Restaurant,Bakery,Bookstore,Pizza Place


## 5. Conclusions <a class="anchor" id="item6"></a>

As shown above, the Toronto neighborhood, Little Portugal, Trinity, could have potential to be selected as a candidate to open a Sushi restaurant, because it shares similar venue patterns with their counterparts in New York City.